In [1]:
import numpy as np
import plotly.express as px
from instance_mongodb import instance_mongodb_sei

from pymatgen.core.structure import Molecule

import torch

from e3nn import o3

In [2]:
def rotate_three_dimensions(alpha, beta, gamma):
    """Rotate the molecule by arbitrary angles alpha
    beta and gamma."""
    cos = np.cos
    sin = np.sin

    r_matrix = [
        [
            cos(alpha) * cos(beta),
            cos(alpha) * sin(beta) * sin(gamma) - sin(alpha) * cos(gamma),
            cos(alpha) * sin(beta) * cos(gamma) + sin(alpha) * sin(gamma),
        ],
        [
            sin(alpha) * cos(beta),
            sin(alpha) * sin(beta) * sin(gamma) + cos(alpha) * cos(gamma),
            sin(alpha) * sin(beta) * cos(gamma) - cos(alpha) * sin(gamma),
        ],
        [-sin(beta), cos(beta) * sin(gamma), cos(beta) * cos(gamma)],
    ]

    r_matrix = np.array(r_matrix)

    return r_matrix

In [10]:
# Visualise computed Hamiltonian
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_waters_dataset

hamiltonians = []
angles = []
structures = []

for doc in collection.find({}):
    hamiltonians.append(doc["fock_matrices"][0])
    structure = Molecule.from_dict(doc["structures"][0])
    structures.append(structure)
    angles.append(doc["angles"])

hamiltonians = np.array(hamiltonians)
angles = np.array(angles)

print(f"Shape of hamiltonians: {hamiltonians.shape}")

fig = px.imshow(
    hamiltonians[:, 1, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)

fig.show()

Shape of hamiltonians: (100, 2, 7, 7)


In [11]:
irreps_hamiltonian = o3.Irreps("2x0e + 1x1o + 2x0e")
print(f"Dimension of the Hamiltonian space: {irreps_hamiltonian.dim}")

D_matrices = []
for idx, angle in enumerate(angles):
    alpha, beta, gamma = angle

    rotation_matrix = rotate_three_dimensions(alpha, beta, gamma)
    rotation_matrix = torch.tensor(rotation_matrix)
    if idx == 0:
        rotation_matrix_0 = rotation_matrix

    # Reference the rotation matrix to the first one
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T

    D_matrix = irreps_hamiltonian.D_from_matrix(rotation_matrix)
    D_matrices.append(D_matrix)

D_matrices = torch.stack(D_matrices)
# Convert to numpy array
D_matrices = D_matrices.detach().numpy()

fig = px.imshow(
    D_matrices, animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)
fig.show()

Dimension of the Hamiltonian space: 7


In [19]:
# Referencing all the angles to the first angle, compute the Hamiltonian in the rotated basis
hamiltonians_rotated = []
for i in range(len(angles)):
    hamiltonian_rotated = D_matrices[i] @ hamiltonians[0, ..., :, :] @ D_matrices[i].T
    hamiltonians_rotated.append(hamiltonian_rotated)

hamiltonians_rotated = np.array(hamiltonians_rotated)
hamiltonians_rotated_diff = hamiltonians_rotated - hamiltonians

fig = px.imshow(
    hamiltonians_rotated_diff[:, 0, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
fig.show()